In [1]:
import numpy as np
import cv2
import easyocr
from ultralytics import YOLO

In [2]:
#loading custom train recognition easyocr model:
reader = easyocr.Reader(['en'],gpu = False,user_network_directory='custom_example')

Using CPU. Note: This module is much faster with a GPU.


In [3]:
# Load the YOLOv8 model
model = YOLO('yolov8s.pt')

In [4]:
#load the image:
img =cv2.imread("img.jpg")
image = img.copy()
print(f'original_image:{image.shape}')

original_image:(667, 1000, 3)


### setting up Hsv colour range for white and blue colour :


In [5]:
# Define the HSV color ranges for white
white_lower = np.array([0,0,215])
white_upper = np.array([180,255,255]) # (These ranges will detect white)

# Define the HSV color ranges for deep blue
lower_deep_blue = np.array([100, 50, 50])
upper_deep_blue = np.array([140, 255, 255])

In [6]:
# Run YOLOv8 tracking on the frame, persisting tracks between frames
result = model.predict(source=image,save = True,save_crop = True)
model.names


0: 448x640 3 persons, 184.5ms
Speed: 4.0ms preprocess, 184.5ms inference, 2.5ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs\detect\predict


{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

### Applying Non Maximam Supression:

In [7]:
cordinat = []
confe = []

    
for r in (result):
    boxes = r.boxes.cpu().numpy() 
    detection = boxes
    for i in range(len(detection)):
        
        row = detection[i]
        conf = row.conf
        if conf > 0.4:
            track_id = row.id
            
            xyxys =row.xyxy
            
            for xyxy in xyxys:
                x1 = int(xyxy[0])
                y1 = int(xyxy[1])
                x2 = int(xyxy[2])
                y2 = int(xyxy[3])
                box = np.array([x1,y1,x2,y2])
                
                cordinat.append(box)
                confe.append(float(conf))
                
#clean:
boxes_np =np.array(cordinat).tolist()
confidences_np = np.array(confe).tolist()

#now applying the non maximum Supression:
index = cv2.dnn.NMSBoxes(boxes_np,confidences_np, 0.25,0.7).flatten()
print(index)

[0 1 2]


### Now we have to crop all detecting objects with the help of the index: 

In [8]:
crop =[]

for ind in index:
    #extract bounding box:
    x1,y1,x2,y2 = boxes_np[ind]
    roi = image[y1:y2,x1:x2]
    crop.append(roi)

In [9]:
for i in range(len(crop)):
    img_2 = crop[i]
    # print(f'original_image:{img_2.shape}')

    #Convert the image to a floating-point representation
    img_float = img_2.astype(np.float32) / 255.0

    # Increase contrast (you can experiment with the values)
    alpha = 1
    enhanced_img = cv2.multiply(img_float, alpha)

    # Increase brightness (you can experiment with the values)
    beta = 3
    enhanced_img = cv2.add(enhanced_img, beta / 255.0)

    # Clip the values to the valid range [0, 1]
    enhanced_img = np.clip(enhanced_img, 0, 1)

    # Convert back to uint8 for display
    enhanced_img = (enhanced_img * 255).astype(np.uint8)

    # print(f'enhanced_img:{enhanced_img.shape}')

    # Resize the image to a larger size (e.g., 2x)
    resize_factor = 2
    resized_image = cv2.resize(enhanced_img, (enhanced_img.shape[1]*resize_factor, enhanced_img.shape[0]*resize_factor))

    # Apply sharpening to the resized image
    kernel = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])  # Sharpening kernel
    sharpened_image = cv2.filter2D(resized_image, -1, kernel)

    # print(f'sharpened_image:{sharpened_image.shape}')
    



    hsv_img = cv2.cvtColor(img_2, cv2.COLOR_BGR2HSV)
    blurred_image = cv2.bilateralFilter(hsv_img, d=9, sigmaColor=80, sigmaSpace=75)
    
    # Masking the image to find our color
    mask_white = cv2.inRange(blurred_image, white_lower,white_upper)
    mask_deep_blue = cv2.inRange(blurred_image, lower_deep_blue, upper_deep_blue)
    
    #gtting pixcel for white color:
    pixel_count_w = cv2.countNonZero(mask_white)
    #gtting pixcel for blue color:
    pixel_count_b = cv2.countNonZero(mask_deep_blue)
    
                
    if pixel_count_w > pixel_count_b  :
        
        box = boxes_np[i]
        x1, y1, x2,y2 = box
        out = cv2.rectangle(image, (x1, y1), (x2 , y2), (0, 255, 0), 2) #drawing rectangle
        
        img_gray = cv2.cvtColor(sharpened_image, cv2.COLOR_BGR2GRAY)

        results = reader.readtext(img_gray,allowlist= '0123456789')

        for (bbox, text, prob) in results:
            if prob > 0.4:
                if  len(text) == 2:
                    # print(f'jersey_number:{text} \nbbox:{bbox} \nprob:{prob}')
                    cv2.putText(image,text,(x1,y1-10),cv2.FONT_HERSHEY_COMPLEX,0.7,(255,255,255),2)
                    jersey_number = text
                    print( f'jersey_number:{jersey_number} \nprob:{prob}')

jersey_number:12 
prob:0.9934331108154081


In [12]:
# Write the modified image to a new file
cv2.imwrite('output_image.jpg', image)

# showing the output_image:
cv2.imshow("img",out)
cv2.waitKey(0) == 27
cv2.destroyAllWindows()